In [17]:
import pandas as pd

In [11]:
import psycopg2
conn = psycopg2.connect(database = 'ad_model',
            user = 'ads_user',
            password = 'ads_pass',
            host="blaze-ad-model-rds-aur-cls-prod-ind-ap-southeast-1."
            "cluster-cka7qtmp9a6h.ap-southeast-1.rds.amazonaws.com",
            port = 5432)

cur = conn.cursor()

In [15]:
conn.rollback()

In [37]:
try:
    cur.execute('''
        select
            audience_targeting_rule.ad_set_id,
            audience_targeting_rule_clause.rule_type,
            attribute.name,
            attribute_value.name,
            attribute_value.enabled,
            attribute_value.ssai_tag,
            ad_set.name,
            ad_set.impression_target,
            ad_set.priority
        from audience_targeting_rule
        join audience_targeting_rule_clause
        on audience_targeting_rule.id = audience_targeting_rule_clause.audience_targeting_rule_id
        join audience_targeting_rule_clause_value
        on audience_targeting_rule_clause.id = audience_targeting_rule_clause_value.audience_targeting_rule_clause_id
        join attribute
        on audience_targeting_rule_clause.attribute_id = attribute.id
        join attribute_value
        on audience_targeting_rule_clause_value.attribute_value_id = attribute_value.id
        join ad_set
        on ad_set.id = audience_targeting_rule.ad_set_id
    ''')
    df=pd.DataFrame(cur.fetchall())
except Exception as e:
    print(e)
    conn.rollback()

In [38]:
df.columns = ['ad_set_id', 'rule_type', 'attr_name', 'attr_val', 'attr_enabled', 'ssai', 'ad_set', 'impr_goal', 'priority']
df

,ad_set_id,rule_type,attr_name,attr_val,attr_enabled,ssai,ad_set,impr_goal,priority
0,1,Include,age,31-99,True,A_U,Test goals,1000,2
1,31,Include,state,West Bengal,True,S_WB,CRFINAU19019_Centurion_Priniting_Hindi_WestBen...,1350000,1
2,32,Include,state,Telangana,False,S_TG,CRFINAU19020_Centurion_Aero_Telugu_Telangana_20,1350000,1
3,33,Include,state,Odisha+North East,True,S_ORNE,CRFINAU19050_Centurion_Priniting_Hindi_Orissa_20,1350000,1
4,36,Include,state,Andhra Pradesh,False,S_AP,CRFINAU19051_Centurion_Aero_Telugu_AndhraPrade...,1350000,1
...,...,...,...,...,...,...,...,...,...
24743,17040,Include,platform,Android,True,P_P_AN,Sports_LLC23_Promo_SVOD_English+Hindi_WTC2023_...,10000000,4
24744,17040,Include,platform,iOS,True,P_P_IOS,Sports_LLC23_Promo_SVOD_English+Hindi_WTC2023_...,10000000,4
24745,17041,Include,platform,AndroidTv and FireStick,True,P_P_ANT,Sports_INDAUS23_Promo_SVOD_English+Hindi_WTC20...,15000000,4
24746,17041,Include,platform,Android,True,P_P_AN,Sports_INDAUS23_Promo_SVOD_English+Hindi_WTC20...,15000000,4


In [29]:
df.nunique()

rule_id         8441
ad_set_id       8441
rule_type          2
attr_name         20
attr_val          96
attr_enabled       2
ssai              97
ad_set          7635
impr_goal       3169
priority           5
dtype: int64

In [34]:
df[df.ad_set.duplicated()&~df.ad_set_id.duplicated()].sort_values('ad_set')

,rule_id,ad_set_id,rule_type,attr_name,attr_val,attr_enabled,ssai,ad_set,impr_goal,priority
23771,16638,16439,Include,devicePrice,Device price 15k +,True,D_15T150,1,1,3
24551,16941,16976,Exclude,state,Punjab + Haryana +Himachal Pradesh +Jammu and ...,True,S_PBHRHPJK,1,1,3
20329,15125,15125,Include,platform,Android,True,P_P_AN,1XBET_1XBET_Youraj_Mobile_1MnplusTowns_10,600000,3
20338,15126,15126,Include,city,Kolkata,True,M_KOL,1XBET_1XBET_Youraj_Mobile_1MnplusTowns_10,600000,3
22124,15746,15746,Include,platform,Android,True,P_P_AN,ABFRL_AllenSolly_CoupleTux_Mobile_ROS_20,3208333,3
...,...,...,...,...,...,...,...,...,...,...
6062,6004,6004,Include,state,MaharashtraX,False,S_MHX,teachers_MHTL_30,10000000,2
10590,10515,10515,Include,platform,Android,True,P_P_AN,test,10526316,3
10595,10517,10517,Include,platform,Android,True,P_P_AN,test,6315789,3
10607,10518,10518,Include,platform,Android,True,P_P_AN,test,100,3


In [42]:
df2 = df[~df.ad_set.str.lower().str.contains('test')]
df2

,ad_set_id,rule_type,attr_name,attr_val,attr_enabled,ssai,ad_set,impr_goal,priority
1,31,Include,state,West Bengal,True,S_WB,CRFINAU19019_Centurion_Priniting_Hindi_WestBen...,1350000,1
2,32,Include,state,Telangana,False,S_TG,CRFINAU19020_Centurion_Aero_Telugu_Telangana_20,1350000,1
3,33,Include,state,Odisha+North East,True,S_ORNE,CRFINAU19050_Centurion_Priniting_Hindi_Orissa_20,1350000,1
4,36,Include,state,Andhra Pradesh,False,S_AP,CRFINAU19051_Centurion_Aero_Telugu_AndhraPrade...,1350000,1
5,46,Include,gender,Male,False,G_M,PRINAU19045_VIVOIPL_BUMRAH_STR_25_THE_23,4000000,5
...,...,...,...,...,...,...,...,...,...
24743,17040,Include,platform,Android,True,P_P_AN,Sports_LLC23_Promo_SVOD_English+Hindi_WTC2023_...,10000000,4
24744,17040,Include,platform,iOS,True,P_P_IOS,Sports_LLC23_Promo_SVOD_English+Hindi_WTC2023_...,10000000,4
24745,17041,Include,platform,AndroidTv and FireStick,True,P_P_ANT,Sports_INDAUS23_Promo_SVOD_English+Hindi_WTC20...,15000000,4
24746,17041,Include,platform,Android,True,P_P_AN,Sports_INDAUS23_Promo_SVOD_English+Hindi_WTC20...,15000000,4


In [43]:
df2.groupby('ad_set_id').ssai.apply(list)

ad_set_id
31                           [S_WB]
32                           [S_TG]
33                         [S_ORNE]
36                           [S_AP]
45                           [S_KA]
                    ...            
17037    [P_P_ANT, P_P_AN, P_P_IOS]
17038    [P_P_ANT, P_P_AN, P_P_IOS]
17039    [P_P_ANT, P_P_AN, P_P_IOS]
17040    [P_P_ANT, P_P_AN, P_P_IOS]
17041    [P_P_ANT, P_P_AN, P_P_IOS]
Name: ssai, Length: 8263, dtype: object

In [44]:
df2.to_csv('blaze-demand.csv', index=False)

In [45]:
!ls -alh blaze-demand.csv

-rw-rw-r-- 1 hadoop hadoop 2.2M Mar 12 10:45 blaze-demand.csv
